In [ ]:
# Imports
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import random
from datetime import datetime
import math
import json
import training

In [ ]:
# Constants

# Network Structure
CONTEXT_SIZE = 32       # How many other voxels are considered for a training example
EMBEDDING_SIZE = 32     # Dimensionality of the voxel embedding vector
STACKED_LAYERS = 2      # How many times the network structure repeats itself
ATTENTION_HEADS = 2     # Number of heads in each multi-headed attention mechanism

# Training Hyperparameters
CHECK_RADIUS = 7        # How far away voxels can be to be part of a training example
CENTER_FOCUS = 0.3      # How much to focus on picking voxels close to the center of the cube. Must be between 0 and 1.
LEARNING_RATE = 1e-3
TRAINING_EXAMPLES = 500

In [ ]:
# Load voxel palette
# The output is a 255-dimensional vector of probabilities for different colors
# Which 255 colors can be generated is decided by the palette file

# Index 0 is reserved as 'undecided' voxel
# Index 1 is reserved as 'air' voxel
# Index 2-255 are colors. So there are 254 possible colors.
with open('data/palette.json', 'r') as json_file:
    palette = json.load(json_file)['colors']
    palette_size = len(palette)

print(f"Palette has {palette_size} colors")


In [ ]:
# Create model
def main_model():
    input = keras.Input(shape=(CONTEXT_SIZE, EMBEDDING_SIZE,), name='input_layer')

    x = keras.layers.Dense(EMBEDDING_SIZE, name=f'feedforward_start')(input)

    # Stacked layers
    for i in range(STACKED_LAYERS):
        # Multi-headed attention
        fx = keras.layers.MultiHeadAttention(
            num_heads=ATTENTION_HEADS,
            key_dim=EMBEDDING_SIZE,
            name=f'multi_head_attention_{i}',
        )(x, x, use_causal_mask=True)

        # Residual connection
        x = keras.layers.Add(name=f'residual_connection_{i}a')([x,fx])

        # Normalization
        x = keras.layers.LayerNormalization(name=f'normalization_{i}a')(x)

        # Feedforward
        fx = keras.layers.Dense(EMBEDDING_SIZE, name=f'feedforward_{i}')(x)
        fx = keras.layers.LeakyReLU(name=f'relu_{i}')(fx)

        # Residual connection
        x = keras.layers.Add(name=f'residual_connection_{i}b')([x,fx])

        # Normalization
        x = keras.layers.LayerNormalization(name=f'normalization_{i}b')(x)
    
    # Final feedforward layer
    # Output size should be palette_size-1, since we don't want it to be able to choose "undecided"
    x = keras.layers.Dense(palette_size-1, name='feedforward_final')(x)

    # Softmax
    x = keras.layers.Softmax(name='softmax')(x)
    
    # Build and return model
    return keras.Model(inputs=input, outputs=x)

model = main_model()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:
# Set up loss and optimizer
loss_function = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(LEARNING_RATE)

In [ ]:
# Design training examples
training_examples = training.generate_training_examples(TRAINING_EXAMPLES, CONTEXT_SIZE)

print(f"{len(training_examples)} training examples created.")

print(random.choice(training_examples))

In [ ]:
def encode_training_input(example):
    inputEntry = []
    for index, voxel in enumerate(example):
        inputEntry.append(training.embed(index, voxel[0], voxel[1], palette, EMBEDDING_SIZE))
    if len(inputEntry) < CONTEXT_SIZE:
        zero_elem = [0,] * EMBEDDING_SIZE
        inputEntry += [zero_elem,] * (CONTEXT_SIZE - len(inputEntry))

    return inputEntry

def encode_training_output(example):
    outputEntry = []
    for voxel in example:
        outputEntry.append(training.encode_one_hot(voxel[1], palette_size))
    return outputEntry

# Reformat training examples into tensor format
def encode_training_examples(training_examples):
    training_inputs = []
    training_outputs = []
    for example in training_examples:
        inputEntry = encode_training_input(example)
        outputEntry = encode_training_output(example)
        
        # Shift the output
        inputEntry.pop(0)
        outputEntry.pop(-1)

        # Push to training example list
        training_inputs.append(inputEntry)
        training_outputs.append(outputEntry)

    training_input_tensor = tf.Variable(training_inputs, tf.float64)
    training_output_tensor = tf.Variable(training_outputs, tf.float64)

    return training_input_tensor, training_output_tensor

training_input_tensor, training_output_tensor = encode_training_examples(training_examples)

In [ ]:
# Construct an example sculpture using the model's current progress
# TODO: Move this to a separate file so we can do multithreading and other improvements
def build_sculpture(count, base=None, temperature=1.0):
    # If a base wasn't specified, create one
    # color = int(random.random() * 254) + 1
    color = 182
    voxels = {}
    voxels['0,0,0'] = color
    
    # Build context vector for the sculpture
    context = [((0, 0, 0), color)]

    for i in range(count-1):
        # Determine where the next voxel will go
        # TODO: Encode this data into the model somehow
        next_pos = training.pick_next_voxel(voxels, context)

        # Get the output from the model
        input_data = encode_training_input(context)
        input_tensor = tf.Variable(input_data, tf.float64)
        input_tensor = tf.reshape(input_tensor, [1, -1, EMBEDDING_SIZE])
        output = model(input_tensor, training=False)
        output_probabilities = output[0][len(context)-1]

        # Pick which voxel to generate based on output probabilities
        # TODO: Implement temperature
        choice = random.random()
        chosen_voxel = 1
        for i in range(len(output_probabilities)):
            choice -= output_probabilities[i]
            if choice < 0:
                chosen_voxel = i+1
                break
        
        # Build the voxel
        voxels[training.ttos(next_pos)] = chosen_voxel
        context.append((next_pos, chosen_voxel))
        if len(context) > CONTEXT_SIZE:
            context.pop(0)
        
    return voxels

In [ ]:
# Function for saving sculptures to json
def save_sculpture(sculpture, filename):
    json_data = {
        "size": {
            "x": 5,
            "y": 5,
            "z": 5,
        },
        "voxels": sculpture,
    }
    with open(filename, 'w') as output_file:
        json.dump(json_data, output_file, indent=2)

In [ ]:
# Training step
@tf.function
def train_step(input_data, output_data):
    # Set up tape
    with tf.GradientTape() as tape:
      output = model(input_data, training=True)

      loss = loss_function(output_data, output)

      gradients = tape.gradient(loss, model.trainable_variables)

      optimizer.apply_gradients(zip(gradients, model.trainable_variables))

# Training loop
def train(epochs):
  time_started = datetime.now()
  example_time = 3
  sculptures_made = 0

  # Epochs
  for epoch in range(epochs):
    # Minibatches
    for _ in range(1):
      train_step(training_input_tensor, training_output_tensor)

    # Print status
    # print(f"Completed Epoch {epoch}")

    # Check if we should output an example sculpture
    if (datetime.now()-time_started).total_seconds() >= example_time:
      print("Building example sculpture...")
      sculpture = build_sculpture(125)
      sculptures_made += 1
      sculpture_filename = f"examples/json/example_{(time_started-datetime.utcfromtimestamp(0)).total_seconds()}_{sculptures_made}.json"
      save_sculpture(sculpture, sculpture_filename)
      print("Done")
      example_time = (datetime.now()-time_started).total_seconds() * 1.3

train(10000000)